In [19]:
import os
import pandas as pd

# get all json files that store our interactions with chatgpt
# these are the final, optimized files
json_files_final_run = [
    os.path.join(dp, f) for dp, dn, fn in os.walk("../data/prompts_new_format") for f in fn
    if ("final" in f or "try_4" in f) and f.endswith(".json")]

json_files_final_run += [
    os.path.join(dp, f) for dp, dn, fn in os.walk("../data/prompts_not_used") for f in fn
    if ("final" in f or "try_4" in f) and f.endswith(".json")]

total_files = len(json_files_final_run)

# read all the data into a pandas dataframe
li = []
for filename in json_files_final_run:
    df = pd.read_json(filename)
    li.append(df)
frame_all_runs = pd.concat(li, axis=0, ignore_index=True)

In [20]:
json_files_final_run

['../data/prompts_new_format/prompt4/abbruch1/prompt_4_final_1.json',
 '../data/prompts_new_format/prompt4/versuch8/prompt_4_final_1.json',
 '../data/prompts_new_format/prompt4/versuch6/prompt_4_final_2.json',
 '../data/prompts_new_format/prompt4/versuch1/prompt_4_try_4.json',
 '../data/prompts_new_format/prompt4/versuch7/prompt_4_final_1.json',
 '../data/prompts_new_format/prompt4/versuch9/prompt_4_final_4.json',
 '../data/prompts_new_format/prompt4/versuch10/prompt_4_final_1.json',
 '../data/prompts_new_format/prompt4/versuch2/prompt_4_final_2.json',
 '../data/prompts_new_format/prompt4/versuch5/prompt_4_final_1.json',
 '../data/prompts_new_format/prompt4/versuch4/prompt_4_final_4.json',
 '../data/prompts_new_format/prompt4/versuch3/prompt_4_try_4.json',
 '../data/prompts_new_format/prompt3/versuch8/prompt_3_final_4.json',
 '../data/prompts_new_format/prompt3/versuch6/prompt_3_try_4.json',
 '../data/prompts_new_format/prompt3/versuch1/prompt_3_try_4.json',
 '../data/prompts_new_forma

In [21]:
import json
issues_change_dict = []
for file in json_files_final_run:
    json_data = json.load(open(file))
    iteration_0 = json_data[0]
    iteration_final = json_data[-1]
    iteration_final["number_of_issues"] = "X" if iteration_final["number_of_issues"] == "" else iteration_final["number_of_issues"]
    issues_change_dict.append(
        {"before": iteration_0["number_of_issues"],
         "after": iteration_final["number_of_issues"]}
    )

In [22]:
issues_change_dict

[{'before': '2', 'after': 'X'},
 {'before': '2', 'after': '0'},
 {'before': '2', 'after': 'X'},
 {'before': '2', 'after': '3'},
 {'before': '2', 'after': '0'},
 {'before': '2', 'after': 'X'},
 {'before': '2', 'after': '0'},
 {'before': '2', 'after': 'X'},
 {'before': '2', 'after': 'X'},
 {'before': '2', 'after': 'X'},
 {'before': '2', 'after': '1'},
 {'before': '9', 'after': 'X'},
 {'before': '9', 'after': '16'},
 {'before': '9', 'after': '5'},
 {'before': '9', 'after': '6'},
 {'before': '9', 'after': 'X'},
 {'before': '9', 'after': '5'},
 {'before': '9', 'after': 'X'},
 {'before': '9', 'after': '5'},
 {'before': '9', 'after': 'X'},
 {'before': '9', 'after': 'X'},
 {'before': '0', 'after': '0'},
 {'before': '0', 'after': '0'},
 {'before': '0', 'after': '0'},
 {'before': '0', 'after': '0'},
 {'before': '0', 'after': '0'},
 {'before': '0', 'after': '0'},
 {'before': '0', 'after': '0'},
 {'before': '2', 'after': 'X'},
 {'before': '2', 'after': 'X'},
 {'before': '0', 'after': '0'},
 {'befo

In [23]:
import pandas as pd
df = pd.DataFrame(issues_change_dict)

In [24]:
df

,before,after
0,2,X
1,2,0
2,2,X
3,2,3
4,2,0
...,...,...
87,9,0
88,2,0
89,2,0
90,4,3


In [26]:
df.to_csv(path_or_buf="../data/number_of_issues_before_after.csv", header=True)